# Paquetes

In [1]:
# Básicos:
import re
import os
import pandas as pd
import numpy as np

# Guardaro y carga de archivos:
import json
from joblib import dump, load
import pickle


# Preprocesamiento:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Cross validation:
from sklearn.model_selection import GridSearchCV

# Modelos:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR, LinearSVR
from xgboost import XGBRegressor

# Metricas:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Extra:
import tqdm

# 1. Funciones de preprocesamiento de datos

## 1.1. Filtrado y eliminación de valores nulos:

In [2]:
def aplicar_mascara(df):
    
    df["comunidad"] = df["comunidad"].str.replace("Catalunya", "Cataluña").replace("Andalusia", "Andalucía").replace("Euskadi", "País Vasco")
    
    lista_comunidades = ["Región de Bruselas-Capital", "Distrito de Lisboa", "Jalisco", "State of Rio Grande do Sul", "Inglaterra", "Hanói", "Hồ Chí Minh", "Maryland", "Bogotá", "Baden-Wurtemberg", "Baja Sajonia", "Renania del Norte-Westfalia"]
    comunidades_mask = df["comunidad"].isin(lista_comunidades)
    df= df[~comunidades_mask]
    
    mask = (df["jornada"] == "no especificado") | (df["tipo_contrato"] == "no especificado") | (df["herramientas"] == "set()")
    df_filtrado = df[~mask].dropna()
    df_filtrado = df_filtrado.reset_index(drop= True)
    
    return df_filtrado

## 1.2. Agrupación de categorías:

In [3]:
def agrupador_categorias(df):
    
    # Tipo de contrato
    tipos_contrato = ["indefinido", "temporal", "practicas"]
    df["tipo_contrato"] = df["tipo_contrato"].apply(lambda x: "otros contratos" if x not in tipos_contrato else x)
    
    # Jornada
    jornadas = ["jornada completa", "practicas", "media jornada"]    
    df["jornada"] = df["jornada"].apply(lambda x: "otras jornadas" if x not in jornadas else x)
    
    return df

## 1.3. Encoding a las columnas:

In [4]:
def column_encoder(df, encoder, ruta_encoders):
    
    encoded_columns = ["jornada", "tipo_contrato", "comunidad", "categoria_empleo"]
    for num, columna in enumerate(encoded_columns):
        
        data_encoded = encoder.fit_transform(df[[columna]]).toarray()
            
        with open(ruta_encoders + f"{num}_{columna}_encoder.pickle", 'wb') as archivo:
            pickle.dump(encoder, archivo)
            
        df_encoded = pd.DataFrame(data_encoded, columns= encoder.categories_)
        df = pd.concat([df, df_encoded], axis=1).drop([columna], axis=1)
        
    return df

In [5]:
def tool_encoder(df, encoder, ruta_encoders):
    
    data_encoded = encoder.fit_transform(df['herramientas'])

    with open(ruta_encoders + "4_tool_encoder.pickle", 'wb') as archivo:
        pickle.dump(encoder, archivo)

    df_encoded = pd.DataFrame(data_encoded, columns= encoder.classes_)    
    df = pd.concat([df, df_encoded], axis=1).drop(['herramientas'], axis=1)
    
    return df

## 1.4. Tratamiento de outliers:

In [6]:
def metodo_tukey(df, columna, alfa):
    q1 = df[columna].quantile(0.25)
    q3 = df[columna].quantile(0.75)
    riq = q3 - q1

    df_sin_out = df[df[columna].between(q1 - alfa * riq, q3 + alfa * riq) | (df[columna].isna())]
    
    return df_sin_out

## 1.5. Función de preparación de datos para modelado:

In [7]:
def data_preparator(df, rutas, encoders, log_transf= True):
    
    ########################### Rutas #######################################################
    
    ruta_listas = rutas[0]
    ruta_encoders = rutas[1]
    
    ########################### Selección y limpieza de variables ###########################
    
    df_mod = df.drop(['titulo', 'empresa', 'fecha', 'descripcion','presencialidad', 'solicitudes', 'portal', 'localidad', 'provincia', 'pais', 'latitud', 'longitud'], axis= 1)
    df_mod = aplicar_mascara(df_mod)
    
    ########################### Agrupo categoricas ##########################################
    
    df_mod = agrupador_categorias(df_mod)
    
    ########################### Limpio columna herrramientas ################################
    
    df_mod['herramientas'] = df_mod['herramientas'].apply(lambda x: re.sub(r"[{}']", '', x).split(', '))
    
    ########################### Obtengo listas ##############################################
    
    lista_herramientas = []
    for herramientas in df_mod['herramientas']:
        lista_herramientas.extend(herramientas)
    lista_herramientas = set(lista_herramientas)
    
    lista_jornadas    = [jornada for jornada in df_mod["jornada"].unique()]
    
    lista_experiencia = [exp for exp in range(df_mod["experiencia"].min(), df_mod["experiencia"].max() + 1)]
    
    lista_contratos   = [contrato for contrato in df_mod["tipo_contrato"].unique()]
    
    lista_beneficios  = [True, False]
    
    lista_comunidades = [comunidad for comunidad in df_mod["comunidad"].unique()]
    
    lista_categorias  = [categoria for categoria in df_mod["categoria_empleo"].unique()]
    
    dic_listas = {"herramientas": list(lista_herramientas),
                  "jornada"     : list(lista_jornadas),
                  "experiencia" : list(lista_experiencia),
                  "contrato"    : list(lista_contratos),
                  "beneficios"  : list(lista_beneficios),
                  "comunidades" : list(lista_comunidades),
                  "categorias"  : list(lista_categorias)        
                 }
    ruta_guardado = os.path.join(ruta_listas, "listas.json")

    # Guardar el diccionario como un archivo JSON
    with open(ruta_guardado, 'w') as archivo:
        json.dump(dic_listas, archivo)
    
    ########################### Encoding columnas categoricas ###############################
    
    encoder_columnas = encoders[0]
    df_mod = column_encoder(df_mod, encoder_columnas, ruta_encoders)
    
    encoder_herramientas = encoders[1]
    df_mod = tool_encoder(df_mod, encoder_herramientas, ruta_encoders)
    
    ########################### Elimino outliers del salario maximo y minimo ##################
    
    df_mod = metodo_tukey(df_mod, "salario_min", 1.5)
    df_mod = metodo_tukey(df_mod, "salario_max", 1.5)
    
    ########################### Transformación logarítmica salario maximo y minimo ############
    
    df_mod["salario_min"] = df_mod["salario_min"].apply(np.log)
    df_mod["salario_max"] = df_mod["salario_max"].apply(np.log)
    #df_mod["experiencia"] = df_mod["experiencia"].apply(lambda x: np.log(x+1))
    
    ########################### Limpio nombre columnas ########################################
    
    df_mod.columns = [str(columna).replace("('", "").replace("',)", "") for columna in df_mod.columns]
    
    ########################### Separo dataframes #############################################
    
    # Salario mínimo    
    df_salario_min = df_mod.drop(["salario_max"], axis= 1)
    # Salario máximo
    df_salario_max = df_mod.drop(["salario_min"], axis= 1)
    
    return df_salario_min, df_salario_max

# 2. Funciones de modelado

## 2.1. Función de testeo de modelos de regresión:

In [8]:
def model_tester(modelos, X, y, save= False):
    
    model_cross_holdout = []
    
    for modelo in modelos:

        r2_score_results, mean_squared_error_results, mean_absolute_error_results = [], [], []

        for i in range(20):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
            modelo.fit(X_train, y_train)
            y_pred = modelo.predict(X_test)

            r2_score_results.append(r2_score(y_test, y_pred))
            mean_squared_error_results.append(mean_squared_error(y_test, y_pred))
            mean_absolute_error_results.append(mean_absolute_error(y_test, y_pred))
            

        model_cross_holdout.append([str(modelo).split("(")[0],
                                    np.array(r2_score_results).mean(),
                                    np.array(mean_squared_error_results).mean(),
                                    np.array(mean_absolute_error_results).mean()
                                   ])
        
    

    df_cross_holdout = pd.DataFrame(model_cross_holdout, columns= ["nombre", "mean_r2", "mean_MSE", "mean_MAE"])
    
    if save:
        nombre_archivo = input("Introduce el nombre del archivo: ")
        df_cross_holdout.to_csv(f"{nombre_archivo}.csv", index= False, sep= ",")
    
    return df_cross_holdout

# 2.2. Función de tuning:

In [9]:
def tunning(modelo, parametros, X_train, X_test, y_train, y_test, salario, save= False):
    
    resultados = []
    
    grid_search = GridSearchCV(estimator  = modelo,
                               param_grid = parametros,
                               scoring    = "neg_mean_squared_error",
                               cv         = 5,
                               refit      = True,
                               n_jobs     = -1,  # Use all available processors
                               verbose    = 2)

    model_result = grid_search.fit(X_train, y_train)

    # Mejor modelo:
    best_model = model_result.best_estimator_
    params_best_model = best_model.get_params()

    y_pred = best_model.predict(X_test)

    # Metricas:
    r2 = r2_score(y_test, y_pred)
    MAE = mean_absolute_error(y_test, y_pred)
    MSE = mean_squared_error(y_test, y_pred)

    resultados.append([str(modelo).split("(")[0], best_model, params_best_model, r2, MAE, MSE])
    df_resultados = pd.DataFrame(resultados, columns= ["Nombre", "Modelo", "Parametros","r2_score", "MAE", "MSE"])
    
    if save:
        df_resultados.to_csv(f"salario_tuning_{salario}.csv", index= "False", sep= ",")
        dump(best_model, f'{salario}_model.pkl')
    
    return df_resultados

# 2.3. Función de testeo mejores PCA:

In [10]:
def pca_tester(salarios, modelos_svr, X_sal, y_sal):
    resultados_PCA = []

    for salario, modelo_svr, X, y in zip(salarios, modelos_svr, X_sal, y_sal):

        resultados = []
        for componentes in range(2, X.shape[1]):

            pca = PCA(componentes, random_state=42)
            X_pca = pca.fit_transform(X)
            X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size = 0.2, random_state=42)

            modelo_svr.fit(X_train, y_train)
            y_pred = modelo_svr.predict(X_test)

            # Métricas:
            r2 = r2_score(y_test, y_pred)
            MAE = mean_absolute_error(y_test, y_pred)
            MSE = mean_squared_error(y_test, y_pred)

            if r2 > 0.46:
                resultados.append([salario, componentes, r2, MAE, MSE])

        resultados_PCA.extend(resultados)

    df_resultados_PCA = pd.DataFrame(resultados_PCA, columns= ["salario", "componentes", "R2", "MAE", "MSE"])
    
    return df_resultados_PCA

# 3. Pipeline transformación de datos de entrada

In [11]:
def data_transformer(X, ruta_encoders, ruta_modelos):
    
    #################################### Cargo encoders ###########################################
    
    encoders = [archivo for archivo in os.listdir(ruta_encoders) if archivo.endswith('.pickle')]

    encoders_cargados = {}
    for encoder in encoders:
        ruta_encoder = os.path.join(ruta_encoders, encoder)

        with open(ruta_encoder, 'rb') as file:
            encoders_cargados[encoder] = pickle.load(file)
            
    ###############################################################################################
    
    #################################### Encoding a las columnas ##################################
    
    encoded_columns = ["jornada", "tipo_contrato", "comunidad", "categoria_empleo", "herramientas"]
    
    for columna, encoder in zip(encoded_columns, encoders_cargados.values()):
        
        if columna == "herramientas":
            data_encoded = encoder.transform(X[columna])
            df_encoded = pd.DataFrame(data_encoded, columns= encoder.classes_)    
            X = pd.concat([X, df_encoded], axis=1).drop([columna], axis=1)            
            
        else:            
            data_encoded = encoder.transform(X[[columna]]).toarray()
            df_encoded = pd.DataFrame(data_encoded, columns= encoder.categories_)
            X = pd.concat([X, df_encoded], axis=1).drop([columna], axis=1)
    
    #################################### Limpio nombre columnas ####################################
    
    X.columns = [str(columna).replace("('", "").replace("',)", "") for columna in X.columns]
    
    #################################### Transformación PCA ########################################
    
    with open(ruta_modelos + "pca_min.pickle", 'rb') as file:
        pca_min = pickle.load(file)
        
    with open(ruta_modelos + "pca_max.pickle", 'rb') as file:
        pca_max = pickle.load(file)
        
    X_pca_min = pca_min.transform(X)
    X_pca_max = pca_max.transform(X)
    
    return X_pca_min, X_pca_max